In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import datetime
import pathlib

import numpy
import os
import sklearn.model_selection

from classifier import classification_data

In [8]:
cfg = {
    "fold": 5,
    "n_labels": 5,
    "path_base": "../features",
    "path_out": "out",
    "test_size": 0.2,
    "train_size": 0.8,
    "random_state": 1234
}

In [9]:
surf = numpy.loadtxt("surf64_iwssip.txt")
n_samples, n_features = surf.shape
x_surf, y_surf = surf[0:, 0:n_features - 1], surf[:, n_features - 1]

kf = sklearn.model_selection.KFold(n_splits=cfg["fold"], shuffle=True, random_state=cfg["random_state"])

In [10]:
get_pca = {
    "mobilenetv2": (128, 256, 512, 1024, -1),
    "resnet50v2": (128, 256, 512, 1024, -1),
    "vgg16": (128, 256, 512, -1)
}

for i, file in enumerate(sorted(pathlib.Path(os.path.join(cfg["path_base"], "unet")).rglob("*.npz"))):
    get_source = list(filter(lambda x: x in str(file).lower(), ["unet", "iwssip", "matlab"]))
    source = get_source[0] if len(get_source) > 0 else None
    get_extractor = list(filter(lambda x: x in str(file).lower(), ["mobilenetv2", "vgg16", "resnet50v2"]))
    extractor = get_extractor[0] if len(get_extractor) > 0 else None
    get_dim = list(filter(lambda x: x in str(file).lower(), ["256", "400", "512"]))
    dim = get_dim[0] if len(get_dim) > 0 else None
    get_channel = list(filter(lambda x: x in str(file).lower(), ["rgb", "grayscale"]))
    channel = get_channel[0] if len(get_channel) > 0 else None
    get_orientation = list(filter(lambda x: x in str(file).lower(), ["horizontal", "vertical", "h+v"]))
    orientation = get_orientation[0] if len(get_orientation) > 0 else None
    get_patch = list(filter(lambda x: x in str(file).lower(), ["patch=3", "patch=5", "patch=7"]))
    patch = get_patch[0].split("=")[1] if len(get_patch) > 0 else None

    print(i, len(list(pathlib.Path(os.path.join(cfg["path_base"], "unet")).rglob("*.npz"))))
    print(file)
    print(source, extractor, dim, channel, orientation, patch)

    data = numpy.load(file)
    dataset = extractor
    n_patch = int(patch)
    x = data["features"]
    y = data["labels"]
    n_samples = x.shape[0]
    n_features = x.shape[1]
    x_normalized = sklearn.preprocessing.StandardScaler().fit_transform(x)

    current_datetime = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
    path = os.path.join(cfg["path_out"], current_datetime, dataset)
    pathlib.Path(path).mkdir(parents=True, exist_ok=True)
    for pca in get_pca[dataset]:
        x = x_normalized
        if pca > 0:
            x = sklearn.decomposition.PCA(n_components=pca).fit_transform(x)

        if pca == -1:
            pca = n_features

        classification_data(cfg, dataset, str(file), list(kf.split(x_surf)), pca, n_samples, path, x, y, n_patch=n_patch, orientation=orientation)
        break
        

0 108
../features/unet/GRAYSCALE/256/mobilenetv2/horizontal+patch=3+mobilenetv2.npz
unet mobilenetv2 256 grayscale horizontal 3
0 DecisionTreeClassifier (900, 128) (225, 128)
1 DecisionTreeClassifier (900, 128) (225, 128)
2 DecisionTreeClassifier (900, 128) (225, 128)
3 DecisionTreeClassifier (900, 128) (225, 128)
4 DecisionTreeClassifier (900, 128) (225, 128)
best mean (%): 52.0
best rule: sum, best_std: 0.06902495522957218
best acc (%): 61.3333
best fold: 1, best rule: sum


Exception ignored in: <function TransformNode.set_children.<locals>.<lambda> at 0x7ff402436b80>
Traceback (most recent call last):
  File "/home/xandao/miniconda3/lib/python3.8/site-packages/matplotlib/transforms.py", line 222, in <lambda>
    self, lambda _, pop=child._parents.pop, k=id(self): pop(k))
KeyboardInterrupt: 


KeyboardInterrupt: 